In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from collections import defaultdict

In [2]:
train = pd.read_pickle('data/train_original.pkl')
test = pd.read_pickle('data/test_original.pkl')
special_cols = [col for col in train.columns if train[col].dtype != np.float64]
feature_cols = [col for col in train.columns if col not in special_cols]
target = train.target.values

In [3]:
if False:
    from tqdm import tqdm
    import scipy
    
    class GaussRankScaler():
        def __init__( self ):
            self.epsilon = 1e-9
            self.lower = -1 + self.epsilon
            self.upper =  1 - self.epsilon
            self.range = self.upper - self.lower

        def fit_transform( self, X ):

            i = np.argsort( X, axis = 0 )
            j = np.argsort( i, axis = 0 )

            assert ( j.min() == 0 ).all()
            assert ( j.max() == len( j ) - 1 ).all()

            j_range = len( j ) - 1
            self.divider = j_range / self.range

            transformed = j / self.divider
            transformed = transformed - self.upper
            transformed = scipy.special.erfinv( transformed )
            ############
            # transformed = transformed - np.mean(transformed)

            return transformed

    def transform_freq_feature(df1,df2,df3_base,feat):
        val1=df1[feat].values
        val2=df3_base[feat].values

        defa1=defaultdict(lambda:0)

        for val in val1:
            defa1[str(val)]+=1.
        for val in val2:
            defa1[str(val)]+=1.  

        df1[feat +"_freq"]= df1[feat].apply(lambda x :defa1[str(x)] ) 
        df2[feat+"_freq"]= df2[feat].apply(lambda x :defa1[str(x)] )  

    def load_data():
        train_df = train[feature_cols].copy()
        test_df = test[feature_cols].copy()
        real_test_df = test[feature_cols].copy()

        unique_samples = []
        unique_count = np.zeros_like(test_df)
        for feature in tqdm(range(test_df.shape[1])):
            _, index_, count_ = np.unique(test_df.values[:, feature], return_counts=True, return_index=True)
            unique_count[index_[count_ == 1], feature] += 1

        # Samples which have unique values are real the others are fake
        real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
        synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

        real_test_df=real_test_df.iloc[real_samples_indexes]
        print(real_test_df.shape[0])
        print(len(synthetic_samples_indexes))

        columns=train_df.columns.values
        for col in tqdm(columns):
            transform_freq_feature(train_df,test_df,real_test_df,col)
        '''
        train_df = pd.concat([train_df,
                              pd.read_pickle('features/unique_sum_per_row_train.pkl')], axis=1)
        test_df = pd.concat([test_df,
                              pd.read_pickle('features/unique_sum_per_row_test.pkl')], axis=1)

        for col in tqdm(columns[[6,110, 26, 146, 139, 21, 76, 174, 133, 99, 198, 109, 80, 13, 190, 148, 0, 44, 164]]):
            transform_gmm_feature(train_df,test_df,real_test_df,col)
        ''' 
        print(train_df.isnull().sum().sum(), test_df.isnull().sum().sum(),)
        return train_df, test_df, real_samples_indexes
    
    train_df, test_df, te_real_samples_indexes = load_data()
    
    '''
    for f in tqdm(feature_cols[:200]): # transform into probability
        train_df[f] = 1/(1+np.exp(-train_df[f]))
        test_df[f] = 1/(1+np.exp(-test_df[f]))
    '''   
    
    '''
    for f in tqdm(feature_cols): # normalzie
        vals = train_df[f].append(test_df.loc[te_real_samples_indexes,f]).values
        m, s = vals.mean(), vals.std()
        train_df[f] = (train_df[f]-m)/s
        test_df[f] = (test_df[f]-m)/s
    '''
    for f in tqdm(feature_cols): # normalzie
        vals = GaussRankScaler().fit_transform(train_df[f].append(test_df.loc[te_real_samples_indexes,f]).values.reshape((-1,1)))
        vals = vals[:,0]
        train_df[f] = vals[:train_df.shape[0]]
        test_df.loc[te_real_samples_indexes,f] = vals[train_df.shape[0]:]
        m, s = vals.mean(), vals.std()
        train_df[f] = (train_df[f]-m)/s
        test_df[f] = (test_df[f]-m)/s
        
    print(train_df.isnull().sum().sum(), test_df.isnull().sum().sum(),)
    train_df.to_pickle('data/nn_data_preprocessed_train_v3.pkl')
    test_df.to_pickle('data/nn_data_preprocessed_test_v3.pkl')
else:
    train_df = pd.read_pickle('data/nn_data_preprocessed_train.pkl')
    test_df = pd.read_pickle('data/nn_data_preprocessed_test.pkl')

In [4]:
import pandas as pd
import tensorflow as tf
from keras.preprocessing import text, sequence
import numpy as np
from keras.layers import Input, SpatialDropout1D,Dropout, GlobalAveragePooling1D, GlobalMaxPooling1D, \
                            CuDNNGRU, GRU, Bidirectional, CuDNNLSTM, Dense, Embedding, concatenate, Embedding, \
                            Flatten, Activation, BatchNormalization, regularizers, Conv1D, Conv2D, MaxPooling2D, Reshape, PReLU, \
                            GaussianNoise, Multiply, Add
from keras.constraints import max_norm

from keras.initializers import Orthogonal
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LambdaCallback, Callback, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
import keras.backend as K
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import os
import pickle
import gc; gc.enable()
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
import string
from scipy.stats import boxcox
import re
from sklearn.model_selection import StratifiedKFold, KFold
#from tqdm import tqdm

# check gpu
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4135914352819612748
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10652409037
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4831855576174646784
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
# build model
from keras.layers import Dense, Concatenate, Input, Lambda
from keras.models import Model

from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, activations
from keras.metrics import *

gamma = 2.0
alpha=.25
epsilon = K.epsilon()

def focal_loss(y_true, y_pred):
    pt_1 = y_pred * y_true
    pt_1 = K.clip(pt_1, epsilon, 1-epsilon)
    CE_1 = -K.log(pt_1)
    FL_1 = alpha* K.pow(1-pt_1, gamma) * CE_1
    
    pt_0 = (1-y_pred) * (1-y_true)
    pt_0 = K.clip(pt_0, epsilon, 1-epsilon)
    CE_0 = -K.log(pt_0)
    FL_0 = (1-alpha)* K.pow(1-pt_0, gamma) * CE_0
    
    loss = K.sum(FL_1, axis=1) + K.sum(FL_0, axis=1)
    return loss

def build_model():
    # share components
    inputs = Input(shape=(train_df.shape[1]//2,2))
    main = inputs
    
    main = Dense(32, activation='relu')(main)
    main = Dense(16, activation='relu')(main)
    #main = BatchNormalization()(main)
    main = Flatten()(main)
    
    out = Dense(1, activation = 'sigmoid')(main) # 1 class to be classified

    model = Model(inputs, out)
    model.regularizers = [regularizers.l2(0.0001)]
    
    # use over weights
    model.compile(optimizer = Adam(lr=0.001, clipnorm=1.), loss=focal_loss, metrics=["binary_crossentropy"])
    
    model.summary()
    return model

model = build_model()
K.clear_session()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 2)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 200, 32)           96        
_________________________________________________________________
dense_2 (Dense)              (None, 200, 16)           528       
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3201      
Total params: 3,825
Trainable params: 3,825
Non-trainable params: 0
_________________________________________________________________


In [6]:
# training
from datetime import datetime
from keras_tqdm import TQDMNotebookCallback
from ipywidgets import IntProgress
from sklearn.model_selection import KFold, StratifiedKFold
import warnings; warnings.filterwarnings('ignore') 
from sklearn.metrics import accuracy_score, roc_auc_score

class auc_score_monitor(Callback):
    def __init__(self, val_data, val_target, checkpoint_file, min_lr =1e-5, reduce_lr_patience=2, early_stop_patience=4, factor=0.1):
        self.val_data = val_data
        self.val_target = val_target
        self.checkpoint_file = checkpoint_file
        self.reduce_lr_patience = reduce_lr_patience
        self.early_stop_patience = early_stop_patience
        self.best_val_score = 0
        self.epoch_num = 0
        self.factor = factor
        self.unimproved_lr_counter = 0
        self.unimproved_stop_counter = 0
        self.min_lr = min_lr
        
    def on_train_begin(self, logs={}):
        self.val_scores = []
        
    def on_epoch_end(self, epoch, logs={}):
        val_pred = self.model.predict(self.val_data).reshape((-1,))
        val_score = roc_auc_score(self.val_target, val_pred)
        # clip pred
        self.val_scores.append(val_score)
        
        #print(self.val_target, '\n', val_pred)
        print('Epoch {} val_score: {:.5f}'.format(self.epoch_num, val_score))
        self.epoch_num += 1
        
        if val_score > self.best_val_score:
            print ('Val Score improve from {:5f} to {:5f}'.format(self.best_val_score, val_score))
            self.best_val_score = val_score
            self.unimproved_lr_counter = 0
            self.unimproved_stop_counter = 0
            if self.checkpoint_file is not None:
                self.model.save_weights(self.checkpoint_file)
        else:
            if val_score-self.best_val_score > 1e-4:
                self.unimproved_lr_counter += 1
                self.unimproved_stop_counter += 1
            
        if self.reduce_lr_patience is not None and self.unimproved_lr_counter > self.reduce_lr_patience:
            current_lr = K.eval(self.model.optimizer.lr)
            if current_lr > self.min_lr:
                print('Reduce LR from {:.6f} to {:.6f}'.format(current_lr, current_lr*self.factor))
                K.set_value(self.model.optimizer.lr, current_lr*self.factor)
                self.model.load_weights(self.checkpoint_file)
            else:
                pass
            
            self.unimproved_lr_counter = 0
            
        if self.early_stop_patience is not None and self.unimproved_stop_counter > self.early_stop_patience:
            print('Early Stop Criteria Meet')
            self.model.stop_training = True
                
        return

In [7]:
seed = 0
train_epochs = 50
batch_size=32 # 32 or 64 is good (too huge for my PC), 128 is worse in the past experiments
cpu_count=4  
n_classses = 1
fold_num = 4
model_prefix = 'nn-stf-v9' #'rnn-with-marcus-features-v4'

In [8]:
#from clr_callback import CyclicLR

def special_reshape(vals):
    return np.vstack([v.reshape((2,-1)).T.reshape((1, -1, 2)) for v in vals])

fold = 0  
retrain_fold = None

for tr_ix, val_ix in KFold(fold_num, shuffle=True, random_state=seed).split(target, target):    
    fold += 1

    if retrain_fold is not None and fold not in retrain_fold:
        continue
        
    #if fold < 3: continue
        
    tr = special_reshape(train_df.values[tr_ix,:])
    val = special_reshape(train_df.values[val_ix,:])
    tr_y = target[tr_ix]
    val_y = target[val_ix]
    print(tr.shape, tr_y.shape)

    model = build_model()
    file_path = "model_weights/{}_fold_{}.hdf5".format(model_prefix, fold)

    early_stop = EarlyStopping(patience=4)
    #lr_schd = LearningRateScheduler(lambda epoch: 0.001*(0.2**(epoch//5)), verbose=1)
    lr_schd = ReduceLROnPlateau(factor=0.1, patience=2, min_lr=1e-5, verbose=1)
    check_point = ModelCheckpoint(file_path, monitor = "val_binary_crossentropy", mode = "min", save_best_only = True, verbose = 1)
    wmlog_loss_monitor = auc_score_monitor(val, target[val_ix], 
                                                checkpoint_file=None, reduce_lr_patience=None, early_stop_patience=None, 
                                                factor=None) # calculate weighted m log loss per epoch
    
    history = model.fit(x=tr, y=tr_y,
                        validation_data=(val, val_y),
                        epochs=train_epochs, verbose = 0, batch_size=batch_size,
                        callbacks = [early_stop, lr_schd, check_point,
                                     wmlog_loss_monitor, TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
    
    
    K.clear_session()
    
# input 400-1, dense 16: 1st fold cv, 50 epoch: 0.891156
# (v1) input 200-2, dense 16: cv: CV Mean = 0.91717, Std = 0.00577, all: 0.9172708052064378, lb: 0.919
# (v2) input 200-2, dense 32: cv: CV Mean = 0.92025, Std = 0.00274, all: 0.920259859604747, lb: 0.920
# (v3) input 200-2, dense 64: cv: CV Mean = 0.92017, Std = 0.00308, all: 0.9201596899376223, lb: 
# (v4) input 200-2, dense 32-dense 16: cv: CV Mean = 0.92234, Std = 0.00247, 0.9223124284750207, lb: 
# (v5) input 200-2, dense 32-dense 16-dense 8: cv: CV Mean = 0.92209, Std = 0.00277, 0.9220835129740627, lb:
# (v6) input 200-2, dense 32-dense 16-flat-bn-dense 32: cv: bad, cut off, lb:
# (v7) input 200-2, dense 32-dense 16 (4 fold): cv: CV Mean = 0.92212, Std = 0.00154, 0.9220763307651896, lb: 0.922
# (v8) input 200-2, dense 32-dense 16 (4 fold) + num to prob: cv: 
#          cv: 0.80 first fold, bad..., lb:
# input 200-2, dense 32-dense 16 (4 fold) + rankgauss: # not better (1st fold 0.9185)
# input 200-2, dense 32-dense 16 (4 fold) + class weight {0: 0.55, 1: 5}: # not better (1st fold 0.917-8)
# input 200-2, dense 32-dense 16 (4 fold) + focal loss: # not better (1st fold 0.918)

(150000, 200, 2) (150000,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 2)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 200, 32)           96        
_________________________________________________________________
dense_2 (Dense)              (None, 200, 16)           528       
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3201      
Total params: 3,825
Trainable params: 3,825
Non-trainable params: 0
_________________________________________________________________



Epoch 00001: val_binary_crossentropy improved from inf to 0.26809, saving model to model_weights/nn-stf-v9_fold_1.hdf5
Epoch 0 val_score: 0.90828
Val Score improve from 0.000000 to 0.908276



Epoch 00002: val_binary_crossentropy improved from 0.26809 to 0.25778, saving model to model_weights/nn-stf-v9_fold_1.hdf5
Epoch 1 val_score: 0.91176
Val Score improve from 0.908276 to 0.911762



Epoch 00003: val_binary_crossentropy improved from 0.25778 to 0.24453, saving model to model_weights/nn-stf-v9_fold_1.hdf5
Epoch 2 val_score: 0.91262
Val Score improve from 0.911762 to 0.912621



Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00004: val_binary_crossentropy did not improve from 0.24453
Epoch 3 val_score: 0.91181


...


Epoch 00015: val_binary_crossentropy did not improve from 0.23707
Epoch 14 val_score: 0.92089



Epoch 00016: val_binary_crossentropy did not improve from 0.23707
Epoch 15 val_score: 0.92093
Val Score improve from 0.920896 to 0.920926



Epoch 00017: val_binary_crossentropy did not improve from 0.23707
Epoch 16 val_score: 0.92095
Val Score improve from 0.920926 to 0.920949



Epoch 00018: val_binary_crossentropy did not improve from 0.23707
Epoch 17 val_score: 0.92096
Val Score improve from 0.920949 to 0.920956


KeyboardInterrupt: 

In [ ]:
train_oof = np.zeros((train.shape[0],))
test_oof = np.zeros((test.shape[0],))

train_aucs = []
fold=0
for tr_ix, val_ix in KFold(fold_num, shuffle=True, random_state=seed).split(target, target):    
    fold += 1
    val = special_reshape(train_df.values[val_ix,:])
    val_y = target[val_ix]
    
    model = build_model()
    file_path = "model_weights/{}_fold_{}.hdf5".format(model_prefix, fold)

    # Predict val + test oofs
    model.load_weights(file_path) # load weight with best validation score
    
    pred = model.predict(val, batch_size=batch_size).reshape((len(val_ix),))
    train_oof[val_ix] = pred
    val_auc = roc_auc_score(target[val_ix], pred)
    train_aucs.append(val_auc)
    print('val acc = {:.5f}'.format(val_auc))

    test_oof += model.predict(special_reshape(test_df.values), batch_size=batch_size).reshape((test.shape[0],))/fold_num
    
    K.clear_session()

In [ ]:
print('CV Mean = {:.5f}, Std = {:.5f}'.format(np.mean(train_aucs), np.std(train_aucs)))
full_auc = roc_auc_score(target, train_oof)
print(full_auc)

In [ ]:
pd.to_pickle(train_oof, "oof+submission/{}_fold_{}_seed_{}_oof_train".format(model_prefix, fold_num, seed))
pd.to_pickle(test_oof, "oof+submission/{}_fold_{}_seed_{}_oof_test".format(model_prefix, fold_num, seed))
sub = pd.DataFrame({"ID_code": test.ID_code.values})
sub["target"] = test_oof
sub.to_csv('oof+submission/' + model_prefix + '_' + str(full_auc).replace('.', '_') + ".csv", index=False)